In [46]:
# !pip install langchain-community
# !pip install pypdf
# !pip install sentence_transformers
# !pip install ipywidgets
# !pip install faiss-cpu

In [47]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [48]:
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census/acsbr-015.pdf', 'page': 0})

In [49]:
## Embedding Using Huggingface

huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True} )


In [50]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content))[0])

[-8.46568570e-02 -1.19099114e-02 -3.37892473e-02  2.94559766e-02
  5.19159846e-02  5.73839247e-02 -4.10018042e-02  2.74267718e-02
 -1.05128184e-01 -1.58056095e-02  7.94858187e-02  5.64318709e-02
 -1.31765185e-02 -3.41544300e-02  5.81601448e-03  4.72547710e-02
 -1.30746858e-02  3.12994863e-03 -3.44225951e-02  3.08406260e-02
 -4.09086347e-02  3.52738015e-02 -2.43761316e-02 -4.35831398e-02
  2.41503231e-02  1.31986383e-02 -4.84451558e-03  1.92347076e-02
 -5.43913022e-02 -1.42734960e-01  5.15522761e-03  2.93115620e-02
 -5.60811199e-02 -8.53535440e-03  3.14140804e-02  2.76736151e-02
 -2.06188075e-02  8.24231207e-02  4.15425114e-02  5.79655543e-02
 -3.71587127e-02  6.26162393e-03 -2.41389722e-02 -5.61794965e-03
 -2.51715165e-02  5.04965289e-03 -2.52801627e-02 -2.91948696e-03
 -8.24038219e-03 -5.69605120e-02  2.30823010e-02 -5.54215256e-03
  5.11555932e-02  6.09937571e-02  6.49766698e-02 -5.38514405e-02
  2.19109859e-02 -2.54194718e-02 -4.49223518e-02  4.22459431e-02
  4.75252122e-02  7.23226

In [51]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [53]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [54]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1174beb80> search_kwargs={'k': 3}


In [55]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=""s

In [56]:
from langchain_community.llms import HuggingFaceEndpoint

hf = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-v0.1", max_length=500, temperature=0.1)

query="What is the health insurance coverage?"
hf.invoke(query)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/mohanishverma/.cache/huggingface/token
Login successful


'\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured. The insured agrees to pay the premiums to the insurer.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured. The insured agrees to pay the premiums to the insurer.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured. The insured agrees to pay the premiums to the insurer.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured. The insured agrees 

In [57]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [58]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [59]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [60]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [61]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])